<a href="https://colab.research.google.com/github/thaopham03/evaluating_fan_effects_in_large_language_models/blob/main/Anderson_Experimental_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
import random
import os

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
people_path = './data/People.csv'
places_path = './data/Places.csv'
output_dir = './data/'

inst_preamble = "Following is a list of sentences. "
fact_sep = '. '
fact_format = 'A {} is in the {}'

target_stimuli = ['present', 'absent']

In [59]:
# Load the CSV files
people = pd.read_csv(people_path)['People']
place = pd.read_csv(places_path)['Place']
probabilities_path = os.path.join(output_dir, 'probabilities.csv')
prompts_path = os.path.join(output_dir, 'prompts.csv')
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)



# Function to compute softmax values
def softmax(values):
    exp_values = np.exp(values - np.max(values))
    return exp_values / exp_values.sum()

# Function to generate Gaussian and softmax values for people and places
def generate_probabilities(people, places):
    people_gauss = {person: random.gauss(0, 1) for person in people}
    place_gauss = {place: random.gauss(0, 1) for place in places}
    people_softmax = softmax(np.array(list(people_gauss.values())))
    place_softmax = softmax(np.array(list(place_gauss.values())))
    people_probs = dict(zip(people, people_softmax))
    place_probs = dict(zip(places, place_softmax))
    return people_probs, place_probs



In [60]:
# Function to generate objects set
def generate_objects(people, places, people_probs, place_probs, max_sentences, randomized):
    objects = set()
    if randomized:
        while len(objects) < max_sentences:
            person = random.choices(list(people_probs.keys()), weights=list(people_probs.values()))[0]
            place = random.choices(list(place_probs.keys()), weights=list(place_probs.values()))[0]
            objects.add((person, place))
    else:
        lowercase_letters = 'abcdefghijklmnor'
        uppercase_letters = 'ABCDEFGHIJKLMNOR'
        people_dict = {lowercase_letters[i]: people[i] for i in range(len(lowercase_letters))}
        places_dict = {uppercase_letters[i]: places[i] for i in range(len(uppercase_letters))}
        predefined_pairs = {
            ('a', 'A'), ('b', 'B'), ('c', 'C'), ('d', 'D'), ('e', 'E'), ('f', 'F'), ('g', 'G'), ('h', 'H'),
            ('i', 'I'), ('j', 'J'), ('k', 'K'), ('l', 'L'), ('e', 'K'), ('r', 'R'), ('g', 'J'), ('h', 'R'),
            ('i', 'L'), ('m', 'M'), ('n', 'N'), ('o', 'O'), ('d', 'M'), ('r', 'N'), ('f', 'O'), ('g', 'M'),
            ('h', 'N'), ('i', 'O')
        }
        objects = {(people_dict[p], places_dict[l]) for p, l in predefined_pairs}
    return objects



# Function to generate prompts
def generate_prompts(objects):
    prompt = inst_preamble + fact_sep.join([fact_format.format(obj[0], obj[1]) for obj in objects])
    return prompt


# Function to print objects and prompts
def print_objects_and_prompts(objects, prompt):
    print("Objects:", objects)
    print("Prompts:", prompt)





In [61]:
# Function to calculate fan value
def fan(x, objects, index=None):
    if index is not None:
        return sum(1 for obj in objects if obj[index] == x) / len(objects)
    else:
        return sum(1 for obj in objects if x in obj) / len(objects)


# Function to print fan values
def print_fan_values(objects):
    for obj in objects:
        person, plc = obj
        fan_person = fan(person, objects)
        fan_place = fan(plc, objects)
        print('The sentence "{}" has fan values: person: {}, place: {}'.format(fact_format.format(person,plc), fan_person, fan_place))

In [62]:
# Function to generate final data and save to CSV
def generate_and_save_data(people, places, objects, prompt, path, randomized):
    data = []

    base_prompt = f'{prompt}{fact_sep}' + f'Sentence 1 is "{fact_format}". In the previous list, Sentence 1 is '
    for stim in target_stimuli:
        for person in people:
            for place in places:
                fan_person = fan(person, objects, 0 if not randomized else None)
                fan_place = fan(place, objects, 1 if not randomized else None)
                true_category = "present" if (person, place) in objects else "absent"
                # Combine prompt and preamble
                final_sentence = base_prompt.format(person, place)
                data.append([final_sentence, person, fan_person, place, fan_place, stim, true_category])
    new_prompts_df = pd.DataFrame(data, columns=['Preamble', 'Person', 'Fan_Person', 'Place', 'Fan_Place', 'Dependent_Variable', 'True_Category'])
    new_prompts_df.to_csv(path, index=False, sep = '|' )
    print("New Prompts Data:")
    print(new_prompts_df)



In [64]:
# Main function to run experiments
def run_experiment(people_path, places_path, output_dir, max_sentences=50, randomized=True):
    people = pd.read_csv(people_path)['People']
    places = pd.read_csv(places_path)['Place']
    probabilities_path = os.path.join(output_dir, 'probabilities.csv')
    prompts_path = os.path.join(output_dir, 'prompts.csv')
    if randomized:
        people_probs, place_probs = generate_probabilities(people, places)
    else:
        people_probs, place_probs = None, None
    objects = generate_objects(people, places, people_probs, place_probs, max_sentences, randomized)
    prompt = generate_prompts(objects)
    print_objects_and_prompts(objects, prompt)
    print_fan_values(objects)  # Print fan values
    new_prompts_path = os.path.join(output_dir, 'random.csv' if randomized else 'anderson_true.csv')
    generate_and_save_data(people, places, objects, prompt, new_prompts_path, randomized)

In [65]:
# Parameters
os.makedirs(output_dir, exist_ok=True)

# Run both experiments
run_experiment(people_path, places_path, output_dir, randomized=True)
run_experiment(people_path, places_path, output_dir, randomized=False)

Objects: {('Clerk', 'Courthouse'), ('Clerk', 'Hotel'), ('Journalist', 'Hotel'), ('Pastor', 'Courthouse'), ('Chef', 'Police Station'), ('Pastor', 'Hotel'), ('Writer', 'Hotel'), ('Builder', 'Hotel'), ('Clerk', 'Bank'), ('Doctor', 'Studio'), ('Artist', 'Police Station'), ('Doctor', 'Church'), ('Pastor', 'Bank'), ('Pastor', 'Restaurant'), ('Writer', 'Bank'), ('Chef', 'Church'), ('Pastor', 'University'), ('Architect', 'Farm'), ('Doctor', 'Library'), ('Clerk', 'Police Station'), ('Pharmacist', 'Hotel'), ('Journalist', 'Police Station'), ('Artist', 'Office Building'), ('Clerk', 'Laboratory'), ('Doctor', 'Park'), ('Clerk', 'Studio'), ('Clerk', 'Church'), ('Librarian', 'Bank'), ('Journalist', 'Studio'), ('Architect', 'Laboratory'), ('Doctor', 'Hotel'), ('Journalist', 'Church'), ('Pastor', 'Church'), ('Clerk', 'Construction Site'), ('Clerk', 'Library'), ('Writer', 'Pharmacy'), ('Doctor', 'Bank'), ('Doctor', 'University'), ('Electrician', 'Studio'), ('Police Officer', 'Restaurant'), ('Teacher', '